**Hello**

In [ ]:
## XgBoost, SVM
## json file
# Earlystopping
# mfcc , mel_spect, mfcc+mel_spect(optimal)
# cut_if _necessary , pad_if_necessary
## ReduceLronPlataue ,Learning rate
# BiLSTM

In [ ]:
import numpy as np
import pandas as pd
import librosa
import tensorflow as tf
import os
from tqdm import tqdm

In [ ]:
audio_file_path='/content/drive/MyDrive/HMR_1.0/audio/01_20001_02_Raag_Multani.wav'
librosa_audio_data,librosa_sample_rate=librosa.load(audio_file_path)

In [ ]:
print(librosa_audio_data)

In [ ]:
mfccs = librosa.feature.mfcc(y=librosa_audio_data, sr=librosa_sample_rate, n_mfcc=40)
print(mfccs.shape)

In [ ]:
mfccs

In [ ]:
audio_dataset_path='/content/drive/MyDrive/HMR_1.0/audio'
df=pd.read_excel('/content/drive/MyDrive/HMR_1.0/HMR_dataframe.xlsx')
df.head()

,SL. No.,Name,Taal
0,1,01_21001_1_03_Raga_-_Bilaskhani_Todi,ektaal
1,2,02_21002_1_01_Raga_Ahiri_Todi,ektaal
2,3,03_21003_1_01_Raga_Maru_Bihag,ektaal
3,4,04_21004_3_02_Raga_Darbari_Kanada,ektaal
4,5,05_21005_1_01_Raga_Deshi,ektaal


In [ ]:
def features_extractor(file):
    audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=13)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    
    return mfccs_scaled_features

In [ ]:
extracted_features=[]
for index_num,row in tqdm(df.iterrows()):
    file_name = os.path.join(os.path.abspath(audio_dataset_path),str(row["Name"])+'.wav')
    final_class_labels=row["Taal"]
    data=features_extractor(file_name)
    extracted_features.append([data,final_class_labels])

151it [06:01,  2.39s/it]


In [ ]:
extracted_features

In [ ]:
extracted_features_df=pd.DataFrame(extracted_features,columns=['feature','class'])
extracted_features_df.head()

,feature,class
0,"[-181.5722, 131.79831, -16.336624, 12.027548, ...",ektaal
1,"[-242.85864, 97.61924, -4.3341885, 23.805813, ...",ektaal
2,"[-260.76398, 154.89009, -51.46725, 24.891666, ...",ektaal
3,"[-207.0644, 97.99254, -9.440074, 26.855722, -1...",ektaal
4,"[-225.81985, 126.66821, -24.241718, 17.877567,...",ektaal


In [ ]:
### Split the dataset into independent and dependent dataset
X=np.array(extracted_features_df['feature'].tolist())
y=np.array(extracted_features_df['class'].tolist())

In [ ]:
X.shape

(151, 13)

In [ ]:
y

In [ ]:
#Label Encoding
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(y))

In [ ]:
y

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
X_train.shape

(120, 13)

In [ ]:
y_train.shape

(120, 4)

In [ ]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

In [ ]:
#no. of classes
num_labels=y.shape[1]

In [ ]:
num_labels

4

In [ ]:
model=Sequential()
###first layer
model.add(Dense(512,input_shape=(13,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###second layer
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###third layer
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))

###final layer
model.add(Dense(num_labels))
model.add(Activation('softmax'))

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer=optimizer)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime 

num_epochs = 500
num_batch_size = 16

checkpointer = ModelCheckpoint(filepath='saved_models/audio_classification.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

In [ ]:
test_accuracy=model.evaluate(X_test,y_test,verbose=0)
print(test_accuracy[1])

0.25806450843811035
